In [1]:
import pandas as pd
df = pd.read_csv('./data/wordsim353_agreed.txt', delimiter = "\t", header=None)
df.columns = ['n', 'word1', 'word2', 'org_sim']

In [2]:
df

,n,word1,word2,org_sim
0,t,love,sex,6.77
1,h,tiger,cat,7.35
2,i,tiger,tiger,10.00
3,t,book,paper,7.46
4,M,computer,keyboard,7.62
...,...,...,...,...
347,t,shower,flood,6.03
348,t,weather,forecast,8.34
349,t,disaster,area,6.25
350,t,governor,office,6.34


In [3]:
# mm_model_path = 'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
# bm_model_path = 'benchmark_model_200000_1.0_15v7'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [4]:
mm_model_path = "finetuned_multimodal_lstm"#'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
bm_model_path = 'finetuned_benchmark_lstm'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else "gpu"
from multimodal_model import BenchmarkLSTM, MMLSTM
from multi_bpe import MultiBPE
multi_bpe = MultiBPE()
def load_model(loaded_model, is_benchmark, is_multimodal, train_visual_module):
    if is_benchmark:
        mm_model = BenchmarkLSTM().to(device)
    else:
        mm_model = MMLSTM(is_multimodal=is_multimodal,
                     train_visual_module=train_visual_module).to(device)
    
    mm_model.load_state_dict(torch.load(f'./saved_models/{loaded_model}'))
    mm_model.eval()
    return mm_model

mm_model = load_model(mm_model_path, False, True, False)
bm_model = load_model(bm_model_path, True, False, False)

In [6]:
def cosine_score(w1, w2, model, name='emb_layer.weight'):
    tcs = torch.nn.CosineSimilarity(dim=1)
    e1 = multi_bpe.encode(w1, False, False, None)
    e2 = multi_bpe.encode(w2, False, False, None)
    if len(e1) > 1 or len(e2) > 1:
        a = model.state_dict()[name][e1[0]]
        for idx in e1[1:]:
            a = torch.concat((a, model.state_dict()[name][idx]))
        b = model.state_dict()[name][e2[0]]
        for idx in e2[1:]:
            b = torch.concat((b, model.state_dict()[name][idx]))
        return 999
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
    else:
        a = model.state_dict()[name][e1[0]]
        b = model.state_dict()[name][e2[0]]
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()

def cosine_sim_model(df, model, name):
    l = []    
    for i in range(len(df)):
        w1, w2 = df.word1[i], df.word2[i]
        l.append(cosine_score(w1, w2, model, name))
    return l

In [7]:
df['bm_sim'] = cosine_sim_model(df, bm_model, 'fc.weight')
df['mm_sim'] = cosine_sim_model(df, mm_model, 'fc.weight')

<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()


In [8]:
df = df[df.bm_sim != 999].reset_index(drop=True)
df

,n,word1,word2,org_sim,bm_sim,mm_sim
0,t,love,sex,6.77,0.825354,0.839019
1,h,tiger,cat,7.35,0.862202,0.862343
2,i,tiger,tiger,10.00,1.000000,1.000000
3,t,book,paper,7.46,0.905169,0.893874
4,M,computer,keyboard,7.62,0.903217,0.906386
...,...,...,...,...,...,...
269,t,space,world,6.53,0.854344,0.853353
270,t,preservation,world,6.19,0.833557,0.845442
271,t,disaster,area,6.25,0.845204,0.837734
272,t,governor,office,6.34,0.822488,0.825813


In [9]:
df.corr(method='pearson')

,org_sim,bm_sim,mm_sim
org_sim,1.000000,0.113767,0.079804
bm_sim,0.113767,1.000000,0.908737
mm_sim,0.079804,0.908737,1.000000


In [10]:
import pingouin as pg

#find partial correlation between hours and exam score while controlling for grade
pg.partial_corr(data=df, x='mm_sim', y='org_sim', covar='bm_sim')

,n,r,CI95%,p-val
pearson,274,-0.056869,"[-0.17, 0.06]",0.349237


In [11]:
0.153084**2

0.023434711056

In [12]:
0.195325**2

0.038151855625

In [13]:
0.173956**2

0.030260689936